## Load and inspect data
Load pickle file and inspect contents

In [ ]:
# Import necessary pyologger utilities
from pyologger.utils.folder_manager import *
from pyologger.utils.event_manager import *
from pyologger.plot_data.plotter import *
from pyologger.calibrate_data.zoc import *
from pyologger.io_operations.base_exporter import *

dataset_id = "oror-adult-orca_hr-sr-vid_sw_JKB-PP"
deployment_id = "2023-10-18_oror-001"

#dataset_id = "mian-juv-nese_sleep_lml-ano_JKB"
#deployment_id = "2019-10-25_mian-001"

# Load important file paths and configurations
config, data_dir, color_mapping_path, montage_path = load_configuration()
# Streamlit load data
animal_id, dataset_id, deployment_id, dataset_folder, deployment_folder, data_pkl, param_manager = select_and_load_deployment(
    data_dir, dataset_id=dataset_id, deployment_id=deployment_id
    )
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

In [ ]:
data_pkl_oror = data_pkl
# Import necessary pyologger utilities
from pyologger.utils.folder_manager import *
from pyologger.utils.event_manager import *
from pyologger.plot_data.plotter import *
from pyologger.calibrate_data.zoc import *
from pyologger.io_operations.base_exporter import *

# dataset_id = "oror-adult-orca_hr-sr-vid_sw_JKB-PP"
# deployment_id = "2023-06-13_oror-002"

dataset_id = "mian-juv-nese_sleep_lml-ano_JKB"
deployment_id = "2019-10-25_mian-001"

# Load important file paths and configurations
config, data_dir, color_mapping_path, montage_path = load_configuration()
# Streamlit load data
animal_id, dataset_id, deployment_id, dataset_folder, deployment_folder, data_pkl, param_manager = select_and_load_deployment(
    data_dir, dataset_id=dataset_id, deployment_id=deployment_id
    )
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

In [ ]:
data_pkl_oror.event_data

In [ ]:
data_pkl_oror.sensor_data['pressure']

In [ ]:
original_units_by_channel = {
    meta['original_name']: meta['unit']
    for sensor in data_pkl_oror.sensor_info.values()
    for meta in sensor['metadata'].values()
}
original_units_by_channel

In [ ]:
data_pkl_oror.sensor_info

In [ ]:
def compare_metadata_structures(meta1, meta2, path="root"):
    """
    Recursively compares two metadata dictionaries to identify structural differences.
    Ignores specific sensor/channel names and values.
    Returns a list of inconsistencies.
    """
    differences = []

    if isinstance(meta1, dict) and isinstance(meta2, dict):
        keys1 = set(meta1.keys())
        keys2 = set(meta2.keys())

        for key in keys1 - keys2:
            differences.append(f"❌ Key '{key}' in {path} missing from second metadata")

        for key in keys2 - keys1:
            differences.append(f"❌ Key '{key}' in {path} missing from first metadata")

        for key in keys1 & keys2:
            subpath = f"{path}.{key}"
            differences.extend(compare_metadata_structures(meta1[key], meta2[key], subpath))

    elif isinstance(meta1, list) and isinstance(meta2, list):
        # Compare lengths and types only
        if len(meta1) != len(meta2):
            differences.append(f"⚠️ List length mismatch at {path}: {len(meta1)} vs {len(meta2)}")
        elif meta1 and meta2 and type(meta1[0]) != type(meta2[0]):
            differences.append(f"⚠️ List type mismatch at {path}: {type(meta1[0])} vs {type(meta2[0])}")

    else:
        if type(meta1) != type(meta2):
            differences.append(f"⚠️ Type mismatch at {path}: {type(meta1)} vs {type(meta2)}")

    return differences

In [ ]:
def audit_metadata_consistency(data1, data2, label1="data1", label2="data2"):
    """
    Compares key metadata structures between two data_pkl-like objects.
    Reports missing keys and type mismatches for:
      - deployment_info
      - logger_info
      - sensor_info
      - derived_info
    """
    top_level_keys = ['deployment_info', 'logger_info', 'sensor_info', 'derived_info']
    all_differences = []

    for key in top_level_keys:
        print(f"\n🔍 Comparing {key}...")

        meta1 = getattr(data1, key, {})
        meta2 = getattr(data2, key, {})

        # If it's a dict of dicts (e.g. logger_info, sensor_info), loop by subkeys
        if isinstance(meta1, dict) and all(isinstance(v, dict) for v in meta1.values()):
            shared_keys = set(meta1.keys()) & set(meta2.keys())
            missing_in_1 = set(meta2.keys()) - set(meta1.keys())
            missing_in_2 = set(meta1.keys()) - set(meta2.keys())

            for k in missing_in_1:
                print(f"❌ {k} missing from {label1}.{key}")
            for k in missing_in_2:
                print(f"❌ {k} missing from {label2}.{key}")

            for subkey in shared_keys:
                diffs = compare_metadata_structures(meta1[subkey], meta2[subkey], path=f"{key}.{subkey}")
                if diffs:
                    print(f"⚠️ Differences found in {key}.{subkey}:")
                    for d in diffs:
                        print("  " + d)
                    all_differences.extend(diffs)

        else:
            # Just compare the full dicts (e.g. deployment_info)
            diffs = compare_metadata_structures(meta1, meta2, path=key)
            if diffs:
                for d in diffs:
                    print("  " + d)
                all_differences.extend(diffs)

    print("\n✅ Audit complete.")
    return all_differences


In [ ]:
audit_metadata_consistency(data_pkl, data_pkl_oror, label1="EVOLOCUS", label2="OROR")
